In [1]:
from landmark_detection.utils import export_places_db, load_names_from_yaml
from landmark_detection.pipeline import Pipeline_Landmark_Detection
import os
import pandas as pd

In [2]:
VERSION = 'app'

# DATABASE
force_rebuild = True
save_every = 500
min_area = 0.0
min_sim_db = 0.0

# PIPELINE
detector_file = "yolov8n-oiv7.pt"
extractor_onnx_file = "cvnet-sg-v" + str(VERSION) + ".onnx"
pipeline_onnx_file = "pipeline-yolo-cvnet-sg-v" + str(VERSION) + ".onnx"
image_dim = (640, 640)
allowed_classes = [41,68,70,74,87,95,113,144,150,158,164,165,193,205,212,224,257,
                              298,310,335,351,354,390,393,401,403,439,442,457,466,489,510,512,
                              514,524,530,531,543,546,554,565,573,580,588,591]
score_thresh = 0.05
iou_thresh = 0.25
scales = [0.7071, 1.0, 1.4142]
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
rgem_pr = 2.5
rgem_size = 5
gem_p = 4.6
sgem_ps = 10.0
sgem_infinity = False
eps = 1e-8
topk = 10
min_sim = 0.70
min_votes = 0.60
remove_inner_boxes = 0.5
join_boxes = True

In [3]:
pipeline = Pipeline_Landmark_Detection(
        detector_file = detector_file,
        extractor_onnx_file = extractor_onnx_file,
        pipeline_onnx_file = pipeline_onnx_file,
        image_dim = image_dim,
        allowed_classes = allowed_classes,
        score_thresh = score_thresh,
        iou_thresh = iou_thresh,
        scales = scales,
        mean = mean,
        std = std,
        rgem_pr = rgem_pr,
        rgem_size = rgem_size,
        gem_p = gem_p,
        sgem_ps = sgem_ps,
        sgem_infinity = sgem_infinity,
        eps = eps,
        topk = topk,
        min_sim = min_sim,
        min_votes = min_votes,
        remove_inner_boxes = remove_inner_boxes,
        join_boxes = join_boxes
)

Creando versión ONNX del preprocess
Instanciando el preprocessor
Creando versión ONNX del detector
Ultralytics 8.3.146  Python-3.11.11 torch-2.7.0+cpu CPU (Intel Core(TM) i5-9300HF 2.40GHz)
YOLOv8n summary (fused): 72 layers, 3,492,527 parameters, 0 gradients, 10.5 GFLOPs

PyTorch: starting from 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 605, 8400) (6.9 MB)

ONNX: starting export with onnx 1.17.0 opset 16...
ONNX: slimming with onnxslim 0.1.54...
ONNX: export success  5.2s, saved as 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.onnx' (13.5 MB)

Export complete (8.1s)
Results saved to C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models
Predict:         yolo predict task=detect model=C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_dete

In [4]:
# Paths
CLASS_NAMES_PATH = os.path.join("oiv7.yaml")
PIPELINE_CONFIG_PATH = os.path.join("configs", "pipeline-yolo-cvnet-sg-v" + str(VERSION))
DATASETS_PATH = os.path.abspath("datasets")
APP_PATH = os.path.join("android_app", "app", "src", "main", "assets")

# Dataset Cáceres
CACERES_LANDMARK_NAMES_PATH = os.path.join(DATASETS_PATH, "caceres", "caceres_landmarks.yaml")
CACERES_PATH = os.path.join(DATASETS_PATH, "caceres", "jpg")
CACERES_CSV = os.path.join(DATASETS_PATH, "caceres", "caceres_dataset.csv")
CACERES_LANDMARKS = os.path.join(DATASETS_PATH, "caceres", "caceres_landmarks.csv")
CACERES_DF = os.path.join(DATASETS_PATH, "caceres", "results", "df_caceres-v" + str(VERSION) + ".pkl")
CACERES_DESC = os.path.join(DATASETS_PATH, "caceres", "results", "desc_caceres-v" + str(VERSION) + ".pkl")
CACERES_PLACES_DB = os.path.join("landmark_detection", "test_data", "test_places_db.pkl")

In [5]:
caceres_dataset = pd.read_csv(CACERES_CSV)
caceres_landmarks = pd.read_csv(CACERES_LANDMARKS)

In [6]:
# Build/load Cáceres dataset
df_result_cac, descriptors_final_cac, places_db_cac = pipeline.build_image_database(
    image_folder = CACERES_PATH,
    df_pickle_path = CACERES_DF,
    descriptor_pickle_path = CACERES_DESC,
    image_place_ids = caceres_dataset,
    return_places_db = True,
    force_rebuild = False,
    save_every = save_every,
    min_area = min_area,
    min_sim = min_sim_db
)

Procesando imágenes: 0it [00:00, ?it/s]


In [7]:
class_names = load_names_from_yaml(CACERES_LANDMARK_NAMES_PATH, as_dict=True)

In [8]:
export_places_db(places_db=places_db_cac, label_map=class_names, output_dir=APP_PATH)